# TensorFlow Parte V - Gráfos y `tf.function`

En los Notebooks previos usabamos TensorFlow en su modo de *eager execution*.

En este modo las operaciones de TensorFlow están ejecutadas por Python, operación por operación, y los resultados están devueltos a Python.

El otro modo es *graph execution* que tiene mejor rendimiento y permite portabilidad fuera de Python.

Gráfos son estructuras de datos que contienen un conjunto de objetos de tipo `tf.Operation` que representan unidades de cómputo, y objetos de tipo `tf.Tensor` que representan los datos que fluyen entre las operaciones.

Se puede visualizar un gráfo con TensorBoard (más tarde). Aquí hay un ejemplo para una red neuronal de dos capas:

![](two-layer-network.png)

Gráfos son muy útiles porque permiten usar modelos de TensorFlow en **varios dispositivos**, **en paralelo** y **rápidamente**.

En la práctica, definimos los modelos con Python, y construimos los gráfos cuando sea necesario.

In [1]:
import tensorflow as tf
import timeit
from datetime import datetime

2023-09-09 16:59:03.628839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-09 16:59:04.605230: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Aprovechando de los gráfos

Se puede crear y correr un gráfo en TensorFlow usando `tf.function`, con una llamada o un decorador. `tf.function` toma una función regular y devuelve una `Function`: una función llamable que contruye gráfos de TensorFlow de la función de Python. Se usa una `Function` en la misma manera como su equivalente en Python.

In [2]:
def una_funcion_regular(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

In [7]:
una_funcion_con_un_grafo = tf.function(una_funcion_regular)

In [8]:
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

In [9]:
valor_orig = una_funcion_regular(x1, y1, b1).numpy()

In [10]:
valor_tf = una_funcion_con_un_grafo(x1, y1, b1).numpy()

In [11]:
assert(valor_orig == valor_tf)

`tf.function` aplica a una función, y todas las funciones que esa función llama.

In [12]:
def funcion_interna(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

In [13]:
@tf.function
def funcion_externa(x):
    y = tf.constant([[2.0], [3.0]])
    b = tf.constant(4.0)
    
    return funcion_interna(x, y, b)

`tf.function` crea un gráfo que incluye las funciones `funcion_externa` y `funcion_interna`.

In [14]:
funcion_externa(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

### Convirtiendo funciones de Python en gráfos

Las partes de una función que son de Python necesitan un paso extra para su conversión en gráfos. `tf.function` utiliza una libreria que se llama *AutoGraph* (`tf.autograph`) para convertir código de Python en un código que genera gráfos.

In [15]:
def relu_simple(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0

In [16]:
tf_relu_simple = tf.function(relu_simple)

In [17]:
print("Primer rama, con gráfo:", tf_relu_simple(tf.constant(1)).numpy())

Primer rama, con gráfo: 1


In [18]:
print("Segunda rama, con gráfo:", tf_relu_simple(tf.constant(-1)).numpy())

Segunda rama, con gráfo: 0


Se puede inspeccionar el código producido por AutoGraph y los gráfos generados directamente:

In [20]:
print(tf.autograph.to_code(relu_simple))

def tf__relu_simple(x):
    with ag__.FunctionScope('relu_simple', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal do_return, retval_
            (do_return, retval_) = vars_

        def if_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = ag__.ld(x)
            except:
                do_return = False
                raise

        def else_body():
            nonlocal do_return, retval_
            try:
                do_return = True
                retval_ = 0
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.converted_call(ag__.ld(tf).greater, (ag__.ld(x), 0), None, fscope), if_bo

In [21]:
print(tf_relu_simple.get_concrete_function(tf.constant(1)).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "Greater/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 0
      }
    }
  }
}
node {
  name: "Greater"
  op: "Greater"
  input: "x"
  input: "Greater/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "cond"
  op: "StatelessIf"
  input: "Greater"
  input: "x"
  attr {
    key: "Tcond"
    value {
      type: DT_BOOL
    }
  }
  attr {
    key: "Tin"
    value {
      list {
        type: DT_INT32
      }
    }
  }
  attr {
    key: "Tout"
    value {
      list {
        type: DT_BOOL
        type: DT_INT32
      }
    }
  

### Polimorfismo: una `Function`, muchos gráfos

Un gráfo `tf.Graph` está especializado a entradas específicas (tensores de `dtype` específicos, objetos con el mismo `id()`).

Cuando se invoca una `Function` con argumentos que no corresponden a un gráfo que ya existe (otros `dtype`, formas distintas) `Function` crea un nuevo gráfo `tf.Graph` especializado a los nuevos argumentos.

La especificación de las entradas a un gráfo `tf.Graph` se denomina su **signatura**.

La `Function` guarda el `tf.Graph` que corresponde a una signatura en una `ConcreteFunction` (una evoltura alrededor de un `tf.Graph`).

In [28]:
@tf.function
def mi_relu(x):
    return tf.maximum(0., x)

In [29]:
print(mi_relu(tf.constant(5.5)))

tf.Tensor(5.5, shape=(), dtype=float32)


In [30]:
print(mi_relu([1, -1])) #Nuevo gráfo creado

tf.Tensor([1. 0.], shape=(2,), dtype=float32)


En el caso abajo la signatura ya existe, así que no crea un nuevo gráfo.

In [31]:
print(mi_relu(tf.constant([3., -3.])))

tf.Tensor([3. 0.], shape=(2,), dtype=float32)


Una `Function` es **poliformica** ya que está respaldada por varios gráfos. Así puede apoyar más tipos de entradas que un sólo gráfo y puede optimizar cada `tf.Graph` para mejor rendimiento.

In [32]:
print(mi_relu.pretty_printed_concrete_signatures())

mi_relu(x)
  Args:
    x: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()

mi_relu(x=[1, -1])
  Returns:
    float32 Tensor, shape=(2,)

mi_relu(x)
  Args:
    x: float32 Tensor, shape=(2,)
  Returns:
    float32 Tensor, shape=(2,)


## Ocupando `tf.function`

En la práctica, puede ser difícil asegurando que `tf.function` funcione correctamente.

### *Graph execution* vs. *Eager execution*

El código en una `Function` se puede ejecutar en modo *eager* o como un gráfo. Por defecto, `Function` ejecuta su código como un gráfo.

In [33]:
@tf.function
def error_promedio_cuadrado(y_verdad, y_pred):
    dif_cuad = tf.pow(y_verdad - y_pred, 2)
    return tf.reduce_mean(dif_cuad)

In [34]:
y_verdad = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred   = tf.random.uniform([5], maxval=10, dtype=tf.int32)

In [35]:
print(y_verdad)

tf.Tensor([8 9 7 7 1], shape=(5,), dtype=int32)


In [36]:
print(y_pred)

tf.Tensor([0 2 9 7 4], shape=(5,), dtype=int32)


In [37]:
error_promedio_cuadrado(y_verdad, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=25>

Para verificar que el gráfo de la `Function` está realizando el mismo cómputo que su función equivalente de Python, se puede ejecutarlo en modo *eager* con `tf.config.run_functions_eagerly(True)`. Este desactiva la capacidad de la `Function` crear y correr gráfos.

In [38]:
tf.config.run_functions_eagerly(True)

In [39]:
error_promedio_cuadrado(y_verdad, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=25>

Se puede activar ejecución de gráfos después.

In [40]:
tf.config.run_functions_eagerly(False)

A veces `Function` tiene un comportamiento distinto en modos *eager* y gráfo. Por ejemplo, la función `print` de Python:

In [41]:
@tf.function
def error_medio_cuadrado(y_verdad, y_pred):
    print("Calculando EMC!")
    dif_cuad = tf.pow(y_verdad - y_pred, 2)
    return tf.reduce_mean(dif_cuad)

In [42]:
err = error_medio_cuadrado(y_verdad, y_pred)

Calculando EMC!


In [43]:
err = error_medio_cuadrado(y_verdad, y_pred)

In [44]:
err = error_medio_cuadrado(y_verdad, y_pred)

El mensaje fue imprimido solamente una vez, a pesar de llamar a la función $3$ veces.

El `print` está ejecutado cuando `Function` corre el código original para crear el gráfo en un proceso conocido como *tracing* (trazamiento).

Trazamiento capta las operaciones en un gráfo, y `print` no está capturado en el gráfo. Después, el gráfo corre sin usar el código de Python.

In [45]:
tf.config.run_functions_eagerly(True)

In [46]:
err = error_medio_cuadrado(y_verdad, y_pred)

Calculando EMC!


In [47]:
err = error_medio_cuadrado(y_verdad, y_pred)

Calculando EMC!


In [48]:
err = error_medio_cuadrado(y_verdad, y_pred)

Calculando EMC!


In [49]:
tf.config.run_functions_eagerly(False)

`print` es un efecto secundario de Python. Hay otras limitaciones en la conversión de funciones a una `Function`. Se puede leer más en la [documentación](https://www.tensorflow.org/guide/function).

### Ejecución no-estricta

En el gráfo solamente las operaciones necesarias para producir los efectos observables están ejecutadas, por ejemplo:

* El valor de retorno de la función


* Efectos secundarios muy bien conocidos:
    * Operaciones de entrada/salida, e.g. `tf.print`
    * Operaciones de *debugging*
    * Mutaciones de `tf.Variable`
    
Este se llama **ejecución no estricta** (*non-strict execution*). En modo *eager*, todas las operaciones se ejecutan.

Hay operaciones que no estarán incluidas en el gráfo porque no producen efectos obervables (por ejemplo, checkeo de errores).

In [50]:
def retorno_no_usado_eager(x):
    tf.gather(x, [1]) # No usado
    return x

In [55]:
tf.gather?

In [52]:
try:
    # Pasamos un tensor con un solo elemento, pero el "gather" en la
    # función busca el valor con elemento [1]
    print(retorno_no_usado_eager(tf.constant([0.0])))
except tf.errors.InvalidArgumentError as e:
    print(f'{type(e).__name__}: {e}')

InvalidArgumentError: {{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[0] = 1 is not in [0, 1) [Op:GatherV2]


In [53]:
@tf.function
def retorno_no_usado_grafo(x):
    tf.gather(x, [1]) # No usado
    return x

In [54]:
print(retorno_no_usado_grafo(tf.constant([0.0])))

tf.Tensor([0.], shape=(1,), dtype=float32)


### Mejores prácticas con `tf.function`

* Intercambiar entre modo *eager* y gráfo durante el desarrollo del código con `tf.config.run_functions_eagerly` para identificar diferencias entre los modos.


* Crear `tf.Variable`s fuera de la función de Python y modificarlas dentro. Lo mismo para objetos que ocupan `tf.Variable`, como `tf.keras.layers`, `tf.keras.Model` y `tf.keras.optimizers`.


* Evitar escribir funciones que dependen de variables externas de Python (aparte de `tf.Variable` y objetos de Keras).


* Es mejor escribir funciones que toman tensores y tipos de TensorFlow como entradas.


* Incluir el mayor porcentaje posible de los cálculos del modelo dentro de un `tf.function` para maximizar el rendimiento.

### Acelerando un código

`tf.function` típicamente mejora el rendimiento del código, pero por cuanto va a depender del tipo de cálculo. Cálculos pequeños probablemente estarán limitados por el costo de llamar al gráfo.

In [22]:
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

In [27]:
def potencia(x, y):
    resultado = tf.eye(10, dtype=tf.dtypes.int32)
    for _ in range(y):
        resultado = tf.matmul(x, result)
    return resultado

In [24]:
print("Eager: ", timeit.timeit(lambda: potencia(x, 100), number=1000), "seconds")

Eager:  5.150919185995008 seconds


In [25]:
potencia_como_grafo = tf.function(power)

In [26]:
print("Gráfo: ", timeit.timeit(lambda: potencia_como_grafo(x, 100), number=1000), "seconds")

Gráfo:  0.8541581010067603 seconds


## Determinar cuando hay trazamiento de una `Function` 

Se puede considerar trazamiento (*tracing*) de una función como su **compilación** por primera vez. Es mejor reducir el número de veces que este ocurre, para mejorar el rendimiento.

Se puede determinar cuando está pasando con el uso de un `print` en el código. Generalmente, el `print` se ejecuta cada vez que hay trazamiento de la función.

In [56]:
@tf.function
def funcion_con_efectos_secundarios_de_python(x):
    print("Trazamiento!")
    return x * x + tf.constant(2)

In [57]:
print(funcion_con_efectos_secundarios_de_python(tf.constant(2)))

Trazamiento!
tf.Tensor(6, shape=(), dtype=int32)


In [58]:
print(funcion_con_efectos_secundarios_de_python(tf.constant(3)))

tf.Tensor(11, shape=(), dtype=int32)


Hay *retracing* cada vez cambiamos el argumento si es un tipo de Python.

In [59]:
print(funcion_con_efectos_secundarios_de_python(2))

Trazamiento!
tf.Tensor(6, shape=(), dtype=int32)


In [60]:
print(funcion_con_efectos_secundarios_de_python(3))

Trazamiento!
tf.Tensor(11, shape=(), dtype=int32)
